In [5]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
# Read recipe inputs
ramen_clusters_named = dataiku.Dataset("ramen_clusters_named")
df = ramen_clusters_named.get_dataframe()

In [14]:
df[df['cluster_labels'].isin(['接客などのサービス', '具材・素材・味'])]

,cluster_labels,words_concat,count
0,接客などのサービス,"営業,時間,08,出迎え,皆さん,暑い,30分,到着,15人,11時,券売機,45分,待ち,...",1283
4,具材・素材・味,"途中,綺麗,スープ,丸鶏,昆布,煮干し,濃い,優しい,これ,旨し,鶏油,若干,多め,クド,最...",2109


In [7]:
v = TfidfVectorizer()

In [0]:
x = v.fit_transform(df['sent'])

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

tf_IDF_scores_df = ramen_clusters_named_df # For this sample code, simply copy input to output


# Write recipe outputs
tf_IDF_scores = dataiku.Dataset("TF_IDF_scores")
tf_IDF_scores.write_with_schema(tf_IDF_scores_df)